In [143]:
import time
import redis
import redis_utils

In [22]:
endpoint = "redis-10483.c299.asia-northeast1-1.gce.cloud.redislabs.com:10483"
hostname = endpoint.split(':')[0]
password = "7imP7aKrC1V631exXxO4sUAjaoxWayS9"
port = int(endpoint.split(':')[1])

In [23]:
redis_conn = redis.Redis(
    host=hostname,
    port=port,
    password=password
)

# String

In [36]:
a_string = "test_input"
redis_conn.set("a_string", a_string)

True

In [37]:
redis_conn.exists("a_string")

1

In [43]:
redis_string = redis_conn.get("a_string")
redis_string, type(redis_string)

(b'test_input', bytes)

In [42]:
redis_string.decode('ascii') == a_string

True

In [44]:
redis_conn.delete("a_string")

1

In [45]:
redis_conn.delete("a_string")

0

## List

In [57]:
redis_conn.delete("a_list")

1

In [73]:
redis_conn.rpush("a_list", 1, 2, 3, 4, 5)

5

In [77]:
redis_list = redis_conn.lrange("a_list", 0, -1)

In [76]:
redis_conn.lindex("a_list", 0)

b'1'

In [62]:
transformed_redis_list = [int(i.decode('ascii')) for i in redis_list]

## Sets

In [80]:
a_set = {'a', 'b', 'c'}
for i in a_set:
    redis_conn.sadd("a_set", i)

In [87]:
redis_conn.sismember("a_set", "a"), redis_conn.sismember("a_set", 5)

(True, False)

In [88]:
redis_set = redis_conn.smembers("a_set")

In [94]:
for _ in redis_set:
    redis_set.add(redis_set.pop().decode('ascii'))

In [95]:
redis_set == a_set

True

## Hashes

In [97]:
for i in range(5):
    redis_conn.hset("a_hash", f"key{i}", i)

In [109]:
redis_hash = redis_conn.hgetall('a_hash')

In [110]:
key_list = list(redis_hash.keys())

In [111]:
for key in key_list:
    redis_hash[key.decode('ascii')] = redis_hash.pop(key).decode('ascii')

In [114]:
redis_conn.hdel('a_hash', 'key4')

1

In [112]:
redis_hash

{'key0': '0', 'key1': '1', 'key2': '2', 'key3': '3', 'key4': '4'}

## sorted sets

In [130]:
sorted_set_dict = {f"element{i}": i + 1 for i in range(5)}

In [131]:
redis_conn.zadd("a_sorted_set", sorted_set_dict)

4

In [135]:
redis_conn.zrangebyscore('a_sorted_set', 0, 6)

b'element0'

In [134]:
redis_conn.zrange('a_sorted_set', 0, 6)

[b'element0', b'element1', b'element2', b'element3', b'element4']

# voting article

In [140]:
ONE_WEEK_IN_SECONDS = 7 * 86400
VOTE_SCORE = 432
ARTICLES_PER_PAGE = 25

In [222]:
def article_vote(conn, user, article):
#     cutoff = time.time() - ONE_WEEK_IN_SECONDS
    cutoff = 10
    if conn.zscore('time:', article) < cutoff:
        return
    article_id = article.partition(':')[-1]
    if conn.sadd('voted:' + article_id, user):
        conn.zincrby('score:', VOTE_SCORE, article)
        conn.hincrby(article, 'votes', 1)
        print('complete')

In [149]:
def transform_binary_hash_to_dict(redis_hash):
    key_list = list(redis_hash.keys())
    for key in key_list:
        redis_hash[key.decode('ascii')] = redis_hash.pop(key).decode('ascii')
    return redis_hash

In [145]:
redis_conn = redis_utils.create_redis_conn()

In [186]:
redis_key = "article:92617"
a_dict = {
    'title': 'Go to statement considered harmful',
    'link': 'http://goo.gl/kZUSu',
    'poster': ' user:83271',
    'time': '1331382699.33',
    'votes': '528'
}
for key, value in a_dict.items():
    redis_conn.hset(redis_key, key, value)

In [161]:
redis_conn.zscore('time:', redis_key)

1.0

In [177]:
redis_conn.zadd('time:', {redis_key: 1000000})

1

In [183]:
redis_conn.zadd('score:', {redis_key: 50})

1

In [264]:
artical = transform_binary_hash_to_dict(redis_conn.hgetall(redis_key))

In [240]:
article_vote(redis_conn, "83271", redis_key)

complete


In [219]:
redis_conn.zincrby('score:', '', redis_key)

53.0

In [232]:
redis_conn.smembers('voted:92617')

{b'16986', b'83271'}

In [239]:
redis_conn.srem('voted:92617', '83271')

1

In [238]:
redis_conn.sadd('voted:'+'92617', '83271')

1

In [268]:
def post_article(conn, user, title, link):
    article_id = str(conn.incr('article:'))
    voted = 'voted:' + article_id
    conn.sadd(voted, user)
    conn.expire(voted, ONE_WEEK_IN_SECONDS)
    now = time.time()
    article = 'article:' + article_id
    print(article)
    conn.hmset(article, {
        'title': title,
        'link': link,
        'poster': user,
        'time': now,
        'votes': 1,
    })
    conn.zadd('score:', {article: now + VOTE_SCORE})
    conn.zadd('time:', {article: now})
    return article_id

In [271]:
article_id = post_article(redis_conn, "user", "test_title", "localhost")

article:6


/Users/yurei/my_env_name/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  


In [272]:
redis_conn.hgetall(f"article:{article_id}")

{b'title': b'test_title',
 b'link': b'localhost',
 b'poster': b'user',
 b'time': b'1664292503.195312',
 b'votes': b'1'}

In [278]:
def get_articles(conn, page, order='score:'):
    start = (page - 1) * ARTICLES_PER_PAGE
    end = start + ARTICLES_PER_PAGE - 1
    ids = conn.zrevrange(order, start, end)
    articles = []
    for a_id in ids:
        article_data = conn.hgetall(a_id)
        article_data['id'] = a_id
        articles.append(article_data)
    return articles

In [279]:
get_articles(redis_conn, 0)

[{b'title': b'test_title',
  b'link': b'localhost',
  b'poster': b'user',
  b'time': b'1664292503.195312',
  b'votes': b'1',
  'id': b'article:6'},
 {b'title': b'test_title',
  b'link': b'localhost',
  b'poster': b'user',
  b'time': b'1664292477.1257899',
  b'votes': b'1',
  'id': b'article:5'},
 {b'title': b'test_title',
  b'link': b'localhost',
  b'poster': b'user',
  b'time': b'1664292455.169195',
  b'votes': b'1',
  'id': b'article:4'},
 {b'title': b'test_title',
  b'link': b'localhost',
  b'poster': b'user',
  b'time': b'1664292414.645739',
  b'votes': b'1',
  'id': b'article:3'},
 {b'title': b'test_title',
  b'link': b'localhost',
  b'poster': b'user',
  b'time': b'1664292248.587447',
  b'votes': b'1',
  'id': b'article:2'},
 {b'title': b'Go to statement considered harmful',
  b'link': b'http://goo.gl/kZUSu',
  b'poster': b' user:83271',
  b'time': b'1331382699.33',
  b'votes': b'530',
  'id': b'article:92617'}]

In [276]:
def add_remove_groups(conn, article_id, to_add=[], to_remove=[]):
    article = 'article:' + article_id
    for group in to_add:
        conn.sadd('group:' + group, article)
    for group in to_remove:
        conn.srem('group:' + group, article)

In [280]:
add_remove_groups(redis_conn, '1', ['test_group1'])

In [277]:
def get_group_articles(conn, group, page, order='score:'):
    key = order + group
    if not conn.exists(key):
        conn.zinterstore(
            key,
            ['group:' + group, order],
            aggregate='max',
        )
    conn.expire(key, 60)
    return get_articles(conn, page, key)

In [282]:
get_group_articles(redis_conn, 'test_group1', 0, '')

[]